# GDAL Day 1

GDAL - Geospatial Data Abstration Library

#### Set up
Use - `opengeo` micromamba env. 
Move to `gdal-tools` folder.

In [2]:
cd /Users/abharathi/Documents/gis_data/gdal-tools/

/Users/abharathi/Documents/gis_data/gdal-tools


In [3]:
ls

1870_southern-india.jpg* geonames/                prism/
batch.py*                landsat8/                spatial_query.gpkg
batch_parallel.py*       london_1m_dsm/           srtm/
earth_at_night.jpg*      naip/                    worldcities.csv*
earthquakes/             precipitation.gpkg


## Intro

GDAL and OGR started as two different programs, hence have a different usage pattern or design.

**Why use GDAL**
 - faster
 - smaller, fits headless, remote execution
 - FOSS
 - great for server-side execution
 - great for compression
 
 
**Speed**
 - GDAL is fast. If we use Python to kick off GDAL as a CLI - it is still fast (`os.system`)
 - If we use Python bindings, they are much slower than using raw GDAL

### Tools with GDAL

We get 2 sets of tools - raster tools that come with GDAL and vector tools that come with OGR.

3 Main tools
 - `gdalinfo` - quick metadata
 - `gdal_translate` - convert formats, apply compression
 - `gdalwarp` - resampling and reprojecting
 
Other tools
 - `gdaladdo` - pyramid tiles for images for quicker GUI loading
 - `gdaltindex` - tile index will give overviews of tiles
 - `gdalbuiltvrt` - for creating virtual rasters
 - `gdaldem`, `gdalcounter` - elevation
 - `gdal_rasterize`, `gdal_poligonize` - for conversion to and from raster <--> vector
 - `gdal_grid` - interpolations
 - `nearblack` - work with edge artifacts in rasters
 
Python bindings
 - `gdal_merge.py` - merge rasters
 - `gdal_calc.py` - raster algebra
 - `gdal_pansharpen.py` - for optical images
 - `gdal_retile.py` - tiling / chipping for deep learning
 
OGR
 - `ogrinfo` - lists source and metadata
 - `ogr2ogr` - format conversion
 - `ogrmerge.py` - merge vector layers

In [5]:
!which gdalinfo

/Users/abharathi/micromamba/envs/opengeo/bin/gdalinfo


In [13]:
!gdalinfo --version

GDAL 3.6.2, released 2023/01/02


In [11]:
ls /Users/abharathi/micromamba/envs/opengeo/bin/ogr*

/Users/abharathi/micromamba/envs/opengeo/bin/ogr2ogr*
/Users/abharathi/micromamba/envs/opengeo/bin/ogr_layer_algebra.py*
/Users/abharathi/micromamba/envs/opengeo/bin/ogrinfo*
/Users/abharathi/micromamba/envs/opengeo/bin/ogrlineref*
/Users/abharathi/micromamba/envs/opengeo/bin/ogrmerge.py*
/Users/abharathi/micromamba/envs/opengeo/bin/ogrtindex*


# Exercises

In [15]:
# formats supported by gdal
!gdalinfo --formats

Supported Formats:
  VRT -raster,multidimensional raster- (rw+v): Virtual Raster
  DERIVED -raster- (ro): Derived datasets using VRT pixel functions
  GTiff -raster- (rw+vs): GeoTIFF
  COG -raster- (wv): Cloud optimized GeoTIFF generator
  NITF -raster- (rw+vs): National Imagery Transmission Format
  RPFTOC -raster- (rovs): Raster Product Format TOC format
  ECRGTOC -raster- (rovs): ECRG TOC format
  HFA -raster- (rw+v): Erdas Imagine Images (.img)
  SAR_CEOS -raster- (rov): CEOS SAR Image
  CEOS -raster- (rov): CEOS Image
  JAXAPALSAR -raster- (rov): JAXA PALSAR Product Reader (Level 1.1/1.5)
  GFF -raster- (rov): Ground-based SAR Applications Testbed File Format (.gff)
  ELAS -raster- (rw+v): ELAS
  ESRIC -raster- (rov): Esri Compact Cache
  AIG -raster- (rov): Arc/Info Binary Grid
  AAIGrid -raster- (rwv): Arc/Info ASCII Grid
  GRASSASCIIGrid -raster- (rov): GRASS ASCII Grid
  ISG -raster- (rov): International Service for the Geoid
  SDTS -raster- (rov): SDTS Raster
  DTED -raster- 

### Basic raster processing

In [17]:
cd srtm/

/Users/abharathi/Documents/gis_data/gdal-tools/srtm


In [19]:
ls -lh

total 202624
-rw-r--r--@ 1 abharathi  staff    25M Sep  7  2020 N27E086.hgt
-rw-r--r--@ 1 abharathi  staff    25M Sep  7  2020 N27E087.hgt
-rw-r--r--@ 1 abharathi  staff    25M Sep  7  2020 N28E086.hgt
-rw-r--r--@ 1 abharathi  staff    25M Sep  7  2020 N28E087.hgt


In [20]:
!gdalinfo N28E086.hgt

Driver: SRTMHGT/SRTMHGT File Format
Files: N28E086.hgt
Size is 3601, 3601
Coordinate System is:
GEOGCRS["WGS 84",
    DATUM["World Geodetic System 1984",
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["geodetic longitude (Lon)",east,
            ORDER[2],
            ANGLEUNIT["degree",0.0174532925199433]],
    ID["EPSG",4326]]
Data axis to CRS axis mapping: 2,1
Origin = (85.999861111111116,29.000138888888888)
Pixel Size = (0.000277777777778,-0.000277777777778)
Metadata:
  AREA_OR_POINT=Point
Corner Coordinates:
Upper Left  (  85.9998611,  29.0001389) ( 85d59'59.50"E, 29d 0' 0.50"N)
Lower Left  (  85.9998611,  27.9998611) ( 85d59'59.50"E, 27d59'59.50"N)
Upper Right (  87.0001389,  29.0001389) ( 87d 0' 0.50"E, 29d 0' 0.

In [25]:
# Can chain command options
!gdalinfo -hist N28E086.hgt

Driver: SRTMHGT/SRTMHGT File Format
Files: N28E086.hgt
       N28E086.hgt.aux.xml
Size is 3601, 3601
Coordinate System is:
GEOGCRS["WGS 84",
    DATUM["World Geodetic System 1984",
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["geodetic longitude (Lon)",east,
            ORDER[2],
            ANGLEUNIT["degree",0.0174532925199433]],
    ID["EPSG",4326]]
Data axis to CRS axis mapping: 2,1
Origin = (85.999861111111116,29.000138888888888)
Pixel Size = (0.000277777777778,-0.000277777777778)
Metadata:
  AREA_OR_POINT=Point
Corner Coordinates:
Upper Left  (  85.9998611,  29.0001389) ( 85d59'59.50"E, 29d 0' 0.50"N)
Lower Left  (  85.9998611,  27.9998611) ( 85d59'59.50"E, 27d59'59.50"N)
Upper Right (  87.0001389,  29.0001389)

### Making virtual rasters
Helps save space. A `.vrt` file is a pointer to the source files and acts like a container.

In [27]:
ls *.hgt > srtm_filelist.txt

In [28]:
cat srtm_filelist.txt

N27E086.hgt
N27E087.hgt
N28E086.hgt
N28E087.hgt


In [29]:
!gdalbuildvrt -input_file_list srtm_filelist.txt merged.vrt

0...10...20...30...40...50...60...70...80...90...100 - done.


In [30]:
cat merged.vrt

<VRTDataset rasterXSize="7201" rasterYSize="7201">
  <SRS dataAxisToSRSAxisMapping="2,1">GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]</SRS>
  <GeoTransform>  8.5999861111111116e+01,  2.7777777777777778e-04,  0.0000000000000000e+00,  2.9000138888888888e+01,  0.0000000000000000e+00, -2.7777777777777778e-04</GeoTransform>
  <VRTRasterBand dataType="Int16" band="1">
    <NoDataValue>-32768</NoDataValue>
    <ComplexSource>
      <SourceFilename relativeToVRT="1">N27E086.hgt</SourceFilename>
      <SourceBand>1</SourceBand>
      <SourceProperties RasterXSize="3601" RasterYSize="3601" DataType="Int16" BlockXSize="3601" BlockYSize="1" />
      <SrcRect xOff="0" yOff="0" xSize="3601" ySize="3601" />
      <DstRect xOff="0" yOff="3600" xSiz

In [31]:
!gdalinfo -stats merged.vrt

Driver: VRT/Virtual Raster
Files: merged.vrt
       N27E086.hgt
       N27E087.hgt
       N28E086.hgt
       N28E087.hgt
Size is 7201, 7201
Coordinate System is:
GEOGCRS["WGS 84",
    DATUM["World Geodetic System 1984",
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["geodetic longitude (Lon)",east,
            ORDER[2],
            ANGLEUNIT["degree",0.0174532925199433]],
    ID["EPSG",4326]]
Data axis to CRS axis mapping: 2,1
Origin = (85.999861111111116,29.000138888888888)
Pixel Size = (0.000277777777778,-0.000277777777778)
Corner Coordinates:
Upper Left  (  85.9998611,  29.0001389) ( 85d59'59.50"E, 29d 0' 0.50"N)
Lower Left  (  85.9998611,  26.9998611) ( 85d59'59.50"E, 26d59'59.50"N)
Upper Right (  88.0001389,  29.0

In [33]:
# Get maximum of the merged rasters
%time
!gdalinfo -stats -json merged.vrt | jq ".bands[0].maximum"

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 9.06 µs
8748


### Converting formats - `gdal_translate`

In [34]:
!gdal_translate --formats

Supported Formats:
  VRT -raster,multidimensional raster- (rw+v): Virtual Raster
  DERIVED -raster- (ro): Derived datasets using VRT pixel functions
  GTiff -raster- (rw+vs): GeoTIFF
  COG -raster- (wv): Cloud optimized GeoTIFF generator
  NITF -raster- (rw+vs): National Imagery Transmission Format
  RPFTOC -raster- (rovs): Raster Product Format TOC format
  ECRGTOC -raster- (rovs): ECRG TOC format
  HFA -raster- (rw+v): Erdas Imagine Images (.img)
  SAR_CEOS -raster- (rov): CEOS SAR Image
  CEOS -raster- (rov): CEOS Image
  JAXAPALSAR -raster- (rov): JAXA PALSAR Product Reader (Level 1.1/1.5)
  GFF -raster- (rov): Ground-based SAR Applications Testbed File Format (.gff)
  ELAS -raster- (rw+v): ELAS
  ESRIC -raster- (rov): Esri Compact Cache
  AIG -raster- (rov): Arc/Info Binary Grid
  AAIGrid -raster- (rwv): Arc/Info ASCII Grid
  GRASSASCIIGrid -raster- (rov): GRASS ASCII Grid
  ISG -raster- (rov): International Service for the Geoid
  SDTS -raster- (rov): SDTS Raster
  DTED -raster- 

In [35]:
%time
# gdal will guess the output format
!gdal_translate merged.vrt merged.tif

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.87 µs
Input file size is 7201, 7201
0...10...20...30...40...50...60...70...80...90...100 - done.


In [36]:
ls -lh merged*

-rw-r--r--  1 abharathi  staff    99M Apr 25 07:15 merged.tif
-rw-r--r--  1 abharathi  staff   2.6K Apr 25 06:58 merged.vrt


In [37]:
%time
# gdal will guess the output format
!gdal_translate merged.vrt merged.zarr

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 11 µs
ERROR 1: Cannot guess driver for merged.zarr
Output driver not found.


In [38]:
!gdalinfo merged.tif

Driver: GTiff/GeoTIFF
Files: merged.tif
Size is 7201, 7201
Coordinate System is:
GEOGCRS["WGS 84",
    ENSEMBLE["World Geodetic System 1984 ensemble",
        MEMBER["World Geodetic System 1984 (Transit)"],
        MEMBER["World Geodetic System 1984 (G730)"],
        MEMBER["World Geodetic System 1984 (G873)"],
        MEMBER["World Geodetic System 1984 (G1150)"],
        MEMBER["World Geodetic System 1984 (G1674)"],
        MEMBER["World Geodetic System 1984 (G1762)"],
        MEMBER["World Geodetic System 1984 (G2139)"],
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]],
        ENSEMBLEACCURACY[2.0]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["geodetic longitude (Lon)",east,
            ORDER[2],
            ANGLEUNIT["degree",0.0174532925199433]],
    USAGE[
  

#### Convert format and compress

Use creation options (`-co`) to specify compression when creating the resulting raser. Doc: https://gdal.org/drivers/raster/gtiff.html#creation-options

`TILDED=YES` - stores tiled TIFF where neighborhood effect is used for compression
`PREDICTOR=YES` - stores diff  in values

In [43]:
%time
!gdal_translate merged.vrt merged.tif -co COMPRESS=DEFLATE -co TILED=YES -co PREDICTOR=2

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 9.78 µs
Input file size is 7201, 7201
0...10...20...30...40...50...60...70...80...90...100 - done.


In [44]:
ls -lh *.tif

-rw-r--r--  1 abharathi  staff    39M Apr 25 07:31 merged.tif


In [45]:
!gdalinfo merged.tif

Driver: GTiff/GeoTIFF
Files: merged.tif
Size is 7201, 7201
Coordinate System is:
GEOGCRS["WGS 84",
    ENSEMBLE["World Geodetic System 1984 ensemble",
        MEMBER["World Geodetic System 1984 (Transit)"],
        MEMBER["World Geodetic System 1984 (G730)"],
        MEMBER["World Geodetic System 1984 (G873)"],
        MEMBER["World Geodetic System 1984 (G1150)"],
        MEMBER["World Geodetic System 1984 (G1674)"],
        MEMBER["World Geodetic System 1984 (G1762)"],
        MEMBER["World Geodetic System 1984 (G2139)"],
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]],
        ENSEMBLEACCURACY[2.0]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["geodetic longitude (Lon)",east,
            ORDER[2],
            ANGLEUNIT["degree",0.0174532925199433]],
    USAGE[
  

#### Assign a different no data value

In [46]:
%time
!gdal_translate merged.vrt merged.tif -co COMPRESS=DEFLATE \
-co TILED=YES -co PREDICTOR=2 -a_nodata -9999

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 10 µs
Input file size is 7201, 7201
0...10...20...30...40...50...60...70...80...90...100 - done.


In [57]:
!gdalinfo -json merged.tif | jq ".bands[0].noDataValue"

-9999


#### Writing GOG
Need to specify output format using `-of COG` as COG also has same `*.tif` extension

In [59]:
%time
!gdal_translate -of COG merged.vrt merged_cog.tif \
  -co COMPRESS=DEFLATE -co PREDICTOR=2 -co NUM_THREADS=ALL_CPUS \
  -a_nodata -9999

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 10 µs
Input file size is 7201, 7201
0...10...20...30...40...50...60...70...80...90...100 - done.


In [60]:
!ls -lh *.tif

-rw-r--r--  1 abharathi  staff    39M Apr 25 07:40 merged.tif
-rw-r--r--  1 abharathi  staff    54M Apr 25 07:50 merged_cog.tif


In [65]:
!gdalinfo merged_cog.tif -json | jq ".metadata.IMAGE_STRUCTURE"

{
  "COMPRESSION": "DEFLATE",
  "INTERLEAVE": "BAND",
  "LAYOUT": "COG",
  "PREDICTOR": "2"
}


### Using GDALINFO on COG files stored on the cloud
I am using Open aerial map site for this: https://map.openaerialmap.org/#/-117.81875610351562,33.76544869849223,9/latest/643b06ca8cae390005a1456f?_k=9krcby

In [68]:
!gdalinfo https://oin-hotosm.s3.amazonaws.com/643b06548cae390005a1456c/0/643b06548cae390005a1456d.tif

Driver: GTiff/GeoTIFF
Files: none associated
Size is 12099, 9659
Coordinate System is:
PROJCRS["WGS 84 / UTM zone 36S",
    BASEGEOGCRS["WGS 84",
        DATUM["World Geodetic System 1984",
            ELLIPSOID["WGS 84",6378137,298.257223563,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4326]],
    CONVERSION["UTM zone 36S",
        METHOD["Transverse Mercator",
            ID["EPSG",9807]],
        PARAMETER["Latitude of natural origin",0,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8801]],
        PARAMETER["Longitude of natural origin",33,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8802]],
        PARAMETER["Scale factor at natural origin",0.9996,
            SCALEUNIT["unity",1],
            ID["EPSG",8805]],
        PARAMETER["False easting",500000,
            LENGTHUNIT["metre",1],
            ID["EPSG",8806]],
        PAR

### DEMs using `gdaldem`

In [70]:
# We specify scale since the vert units are in meters whereas the horz units are in degrees
%time
!gdaldem hillshade merged.vrt hillshade.tif -s 111120

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.82 µs
0...10...20...30...40...50...60...70...80...90...100 - done.


In [71]:
ls -lh hill*

-rw-r--r--  1 abharathi  staff    49M Apr 25 08:00 hillshade.tif


#### DEM classified (color relief) file

In [74]:
# first create a file to store the classes
1000,101,146,82
1500,190,202,130
2000,241,225,145
2500,244,200,126
3000,197,147,117
4000,204,169,170
5000,251,238,253
6000,255,255,255

(6000, 255, 255, 255)

In [77]:
cat colormap.txt

1000,101,146,82
1500,190,202,130
2000,241,225,145
2500,244,200,126
3000,197,147,117
4000,204,169,170
5000,251,238,253
6000,255,255,255

In [78]:
%time
!gdaldem color-relief merged.vrt colormap.txt colorized_dem.tif

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 11.2 µs
0...10...20...30...40...50...60...70...80...90...100 - done.


In [82]:
# Convert to PNG, but a downsampled file that is 10% x 10%
%time
!gdal_translate -of PNG colorized_dem.tif colorized.png -outsize 10% 10%

CPU times: user 5 µs, sys: 3 µs, total: 8 µs
Wall time: 16.9 µs
Input file size is 7201, 7201
0...10...20...30...40...50...60...70...80...90...100 - done.


In [83]:
ls -lh colorized*

-rw-r--r--  1 abharathi  staff   595K Apr 25 08:22 colorized.png
-rw-r--r--  1 abharathi  staff   712B Apr 25 08:22 colorized.png.aux.xml
-rw-r--r--  1 abharathi  staff   148M Apr 25 08:12 colorized_dem.tif


![fie](colorized.png)

In [87]:
0.00009259259259 * 111320

10.3074074071188